<a href="https://colab.research.google.com/github/borja-izquierdo/portfolio/blob/main/Prophet_PYPL_Forecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Forescast of a stock
This is just for ML purposes. Stocks follows a Markovian process which means, the  past could not be related to the future.

In [2]:
!git clone https://github.com/borja-izquierdo/portfolio.git

Cloning into 'portfolio'...
remote: Enumerating objects: 143, done.
remote: Counting objects: 100% (143/143), done.
remote: Compressing objects: 100% (136/136), done.
remote: Total 143 (delta 47), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (143/143), 464.54 KiB | 5.22 MiB/s, done.
Resolving deltas: 100% (47/47), done.


## Install Prophet and Yahoo Finance

In [3]:
# Instalamos el modelo predictivo Prophet
!python -m pip install prophet --quiet
# De aquí vamos a descargar los datos
!pip install yfinance --quiet


## Import necessary libraries

In [4]:
import pandas as pd
import yfinance as yf
import datetime as dt
import matplotlib.pyplot as plt
from datetime import timedelta
import plotly.graph_objects as go
from prophet import Prophet
from prophet.plot import plot_plotly, plot_components_plotly
import warnings

warnings.filterwarnings('ignore')

pd.options.display.float_format = '${:,.2f}'.format

## Set time from to a certain numbers of years

In [5]:
years = 10

endDate = dt.datetime.now()
startDate = endDate - dt.timedelta(days = 365*years)

## Create a ticker variable

In [6]:
ticker = 'PYPL'

## Download the daily data of the ticker

In [7]:
adj_close_df = pd.DataFrame()

adj_close_df =  yf.download(ticker, start = startDate, end = endDate).reset_index()

adj_close_df.tail()

[*********************100%%**********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume
2136,2023-12-28,$62.62,$63.50,$62.45,$63.08,$63.08,11233400
2137,2023-12-29,$63.00,$63.16,$61.15,$61.41,$61.41,16779000
2138,2024-01-02,$61.22,$62.16,$60.43,$61.46,$61.46,15248400
2139,2024-01-03,$60.70,$60.75,$58.35,$58.63,$58.63,21213500
2140,2024-01-04,$58.02,$59.06,$57.44,$58.56,$58.56,9237496


## NAs validation

In [8]:
adj_close_df.isnull().sum()

Date         0
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

## Filter by columns 'Date' & 'Adj Close'

In [9]:
columns =['Date','Adj Close']

new_names = {
    'Date': 'ds',
    'Adj Close':'y',
}
adj_close_df.rename(columns = new_names, inplace=True)
adj_close_df['ds'] = adj_close_df['ds'].dt.tz_localize(None)

## Plot the Historical Adjusted Close Price

In [10]:
x = adj_close_df['ds']
y = adj_close_df['y']

fig = go.Figure()

fig.add_trace(go.Scatter(x=x, y=y))

# Le ponemos el título
fig.update_layout(
    title_text= f'{ticker} Time Series Adjusted Close Price',
)

fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list(
                [
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1a", step="year", stepmode="backward"),
                    dict(step="all"),
                ]
            )
        ),
        rangeslider=dict(visible=True),
        type="date",
    )
)

# Training the model

## Initialize and training with the DataFrame


In [11]:
m= Prophet()
m= Prophet(seasonality_mode='multiplicative')
m = Prophet(daily_seasonality=True, seasonality_mode='multiplicative')


In [12]:
m.fit(adj_close_df)

DEBUG:cmdstanpy:input tempfile: /tmp/tmpw3gf4i32/ya7_he_g.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpw3gf4i32/lw8hc9hm.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=17582', 'data', 'file=/tmp/tmpw3gf4i32/ya7_he_g.json', 'init=/tmp/tmpw3gf4i32/lw8hc9hm.json', 'output', 'file=/tmp/tmpw3gf4i32/prophet_modelu6mtxkyd/prophet_model-20240104174659.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
17:46:59 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
17:47:01 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


### How many days we want to forecast

In [13]:
future = m.make_future_dataframe(60)
future.tail()

,ds
2196,2024-02-29
2197,2024-03-01
2198,2024-03-02
2199,2024-03-03
2200,2024-03-04


### Delete weekends for stocks, ETFs, Forex, Bonds and similars

In [14]:
future = future[ future['ds'].dt.dayofweek < 5 ]

### DataFrame to forecast

In [15]:
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

,ds,yhat,yhat_lower,yhat_upper
2178,2024-02-27,$52.52,$42.62,$63.29
2179,2024-02-28,$52.02,$41.86,$62.92
2180,2024-02-29,$51.65,$41.52,$63.25
2181,2024-03-01,$51.23,$40.02,$61.91
2182,2024-03-04,$49.96,$39.06,$60.83


In [16]:
next_day = (dt.date.today() + timedelta(days=1)).strftime('%Y-%m-%d')

forecast[forecast['ds'] == next_day]['yhat'].item()

59.590424956732946

## Forecast Plot

In [17]:
plot_plotly(m, forecast)

In [18]:
plot_components_plotly(m, forecast)